# Welcome to the [tagbase-server](https://github.com/tagbase/tagbase-server) developer getting started guide

![Tagbase](https://github.com/tagbase/tagbase/wiki/images/tagbase.gif "Tagbase")

# Purpose

This notebook will demonstrate how to user **[pytagbase](https://github.com/tagbase/pytagbase)** to programmatically interface with tagbase-server using the Python programming language.

# Prerequisites

* [Jupyter notebook](https://jupyter.org/install)
* [Docker Compose](https://docs.docker.com/compose/)

# Learning outcomes

By the end of the notebook, you will have learned how to...
* install **[pytagbase](https://github.com/tagbase/pytagbase)**; the Python SDK for tagbase-server
* execute some important functions offered by tagbase-server such as *data ingestion* and *acquiring information about tags*

# More examples and documentation

See the [documentation for API endpoints](https://github.com/tagbase/pytagbase#documentation-for-api-endpoints)

# Support and issue tracking

Please [get in touch with the project community](https://github.com/tagbase/tagbase-server/issues).


In [ ]:
!pip install testcontainers
!pip install git+https://github.com/tagbase/pytagbase.git

In [ ]:
from testcontainers.compose import DockerCompose

with DockerCompose(".", compose_file_name=["docker-compose.yml"]) as compose:
    compose._call_command(['docker-compose', 'build', '--build-arg', 'NGINX_PASS="tagbase"', '--build-arg', 'NGINX_USER="tagbase"', '--build-arg', 'POSTGRES_PASSWORD="tagbase"', '--build-arg', 'POSTGRES_PORT="5432"'])
    compose.start()
    stdout, stderr = compose.get_logs()
    if stderr:
        print("Errors\n:{}".format(stderr))
    
    import pytagbase
    from pytagbase.rest import ApiException
    from pprint import pprint
    
    # Defining the host is optional and defaults to https://localhost/tagbase/api/v0.14.0
    # See configuration.py for a list of all supported configuration parameters.
    configuration = pytagbase.Configuration(
        host = "https://localhost/tagbase/api/v0.14.0",
        password = "tagbase",
        username = "tagbase",
    )
    # Need to bypass self-signed TLS certificates
    configuration.__setattr__("verify_ssl", False)
    
    # Enter a context with an instance of the API client
    with pytagbase.ApiClient(configuration) as api_client:
        # Create an instance of the IngestApi class
        api_instance = pytagbase.IngestApi(api_client)
        file = 'file:///usr/src/app/data/iccat_gbyp0007_ArgosTrans_eTUFF.txt' # str | Location of a network accessible (file, ftp, http, https) file e.g. 'file:///usr/src/app/data/eTUFF-sailfish-117259.txt'.
        notes = 'Ingestion performed via Jupyter Notebook' # str | Free-form text field where details of submitted eTUFF file for ingest can be provided e.g. submitter name, etuff data contents (tag metadata and measurements + primary position data, or just secondary solution-positional meta/data) (optional)
        type = 'etuff' # str | Type of file to be ingested, defaults to 'etuff' (optional) (default to 'etuff')
        version = '1' # str | Version identifier for the eTUFF tag data file ingested (optional)
    
        try:
            # Get network accessible file and execute ingestion
            api_response = api_instance.ingest_get(file)
            print("The response of IngestApi->ingest_get:\n")
            pprint(api_response)
        except ApiException as e:
            print("Exception when calling IngestApi->ingest_get: %s\n" % e)


        # Create an instance of the TagsApi class
        api_instance = pytagbase.TagsApi(api_client)
        try:
            # Get information about all tags
            api_response = api_instance.list_tags()
            print("The response of TagsApi->list_tags:\n")
            pprint(api_response)
        except Exception as e:
            print("Exception when calling TagsApi->list_tags: %s\n" % e)

        tag_id = 1 # int | Existing tag id
        try:
            # Get information about an individual tag
            api_response = api_instance.get_tag(tag_id)
            print("\nThe response of TagsApi->get_tag:\n")
            pprint(api_response)
        except Exception as e:
            print("Exception when calling TagsApi->get_tag: %s\n" % e)
